In [30]:
import numpy as np
import jax
import jax.numpy as jnp
import random
import pandas as pd
import time

In [31]:
def gen_initial_conditions(n):
    matriz = np.zeros((n, n))
    for i in range(n):
        matriz[i, i] = (i+1)/2

    gen_x = jnp.array([1.0 - i/n for i in range(n)])
    return matriz, gen_x


In [32]:
def f_rosenbrock(x):
    x = jnp.array(x)
    total = 0.0
    N = x.shape[0]

    for i in range(0, N-1, 2):
        total += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2

    return total

In [33]:
calculate_grad_rosembrock = jax.grad(f_rosenbrock)
calculate_hess_rosembrock = jax.hessian(f_rosenbrock)

In [ ]:
def newton_method(iterations, x, tol=1e-3):
    x_history = [jnp.array(x[0])]
    for k in range(iterations):
        xk = x_hist[-1]
        gk = calculate_grad_rosembrock(xk)
        Hk = calculate_hess_rosembrock(xk)
        grad_norm = float(jnp.linalg.norm(gk))
        print(f"[Newton] Iter {k} | ||grad|| = {grad_norm:.3e}")
        if grad_norm < tol:
            break
        try:
            p = jnp.linalg.solve(Hk, -gk)
        except Exception as e:
            print(f"[Newton] Hessiana singular na iter {k}: {e}")
            break
        x_next = xk + p
        x_hist.append(x_next)
    return x_hist


In [ ]:
def newton_method_with_metrics(iterations, x, tol=1e-3):
    start_time = time.time()
    x_history = newton_method(iterations, x, tol=tol)
    elapsed_time = time.time() - start_time

    f_values = [float(f_rosenbrock(xi)) for xi in x_history]
    grad_norms = [float(jnp.linalg.norm(calculate_grad_rosembrock(xi))) for xi in x_history]

    iterations_done = len(x_history) - 1

    return {
        'x': x_history,
        'iterations': iterations_done,
        'time': elapsed_time,
        'f_values': f_values,
        'grad_norms': grad_norms,
        'final_f': f_values[-1],
        'final_grad_norm': grad_norms[-1] if grad_norms else 0
    }

In [ ]:
N_values = [1, 10, 100, 1000]
λ_values = [1.0]
results = []

for n in N_values:
    for 𝜆 in 𝜆_values:
        A, x = gen_initial_conditions(n)
        metrics = newton_method_with_metrics(iterations=10000, x=x, tol=1e-3, max_time=800)
        
        results.append({
            'N': n,
            'λ': 𝜆,
            'Iterações': metrics['iterations'],
            'Tempo (s)': metrics['time'],
            'f(x_final)': metrics['final_f'],
            '||∇f(x_final)||': metrics['final_grad_norm'],
            'metrics': metrics
        })

df = pd.DataFrame([{k: v for k, v in r.items() if k != 'metrics'} for r in results])
print("="*70)
print("RESULTADOS - Método de Newton")
print("="*70)
print(df.to_string(index=False))
print("="*70)


=== Rodando N = 1 ===
TFRT_CPU_0


[Newton] Iter 0 | ||grad|| = 0.000e+00
[Newton] Critério de tolerância alcançado.

=== Rodando N = 10 ===
TFRT_CPU_0
[Newton] Iter 0 | ||grad|| = 7.760e+01
[Newton] Iter 1 | ||grad|| = 2.324e+00
[Newton] Iter 2 | ||grad|| = 1.635e+02
[Newton] Iter 3 | ||grad|| = 8.485e-01
[Newton] Iter 4 | ||grad|| = 7.959e+01
[Newton] Iter 5 | ||grad|| = 2.428e-02
[Newton] Iter 6 | ||grad|| = 6.605e-02
[Newton] Iter 7 | ||grad|| = 2.666e-05
[Newton] Critério de tolerância alcançado.

=== Rodando N = 100 ===
TFRT_CPU_0
[Newton] Iter 0 | ||grad|| = 3.623e+02
[Newton] Iter 1 | ||grad|| = 5.297e+02
[Newton] Iter 2 | ||grad|| = 8.261e+02
[Newton] Iter 3 | ||grad|| = 1.750e+03
[Newton] Iter 4 | ||grad|| = 1.816e+02
[Newton] Iter 5 | ||grad|| = 7.110e+01
[Newton] Iter 6 | ||grad|| = 1.443e+01
[Newton] Iter 7 | ||grad|| = 3.233e+01
[Newton] Iter 8 | ||grad|| = 4.468e-03
[Newton] Iter 9 | ||grad|| = 2.239e-03
[Newton] Iter 10 | ||grad|| = 0.000e+00
[Newton] Critério de tolerância alcançado.

=== Rodando N = 10